In [2]:
import netCDF4 as nc
import numpy as np

# Load the data for both years
data_2020 = nc.Dataset('/Users/heyj/Desktop/sql project/2020data.nc')
data_2021 = nc.Dataset('/Users/heyj/Desktop/sql project/2020data.nc')

# Combine data from both years
z_combined = np.concatenate((data_2020['z'][:], data_2021['z'][:]), axis=0)
u_combined = np.concatenate((data_2020['u'][:], data_2021['u'][:]), axis=0)
v_combined = np.concatenate((data_2020['v'][:], data_2021['v'][:]), axis=0)


In [3]:
# Preprocessing the data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()

z_normalized = scaler.fit_transform(z_combined.reshape(-1, 1)).reshape(z_combined.shape)
u_normalized = scaler.fit_transform(u_combined.reshape(-1, 1)).reshape(u_combined.shape)
v_normalized = scaler.fit_transform(v_combined.reshape(-1, 1)).reshape(v_combined.shape)

# Combine the parameters to form a single dataset
data_combined = np.stack((z_normalized, u_normalized, v_normalized), axis=-1)

# Pad the data to get even dimensions
padded_data = np.pad(data_combined, ((0, 0), (0, 1), (0, 1), (0, 1), (0, 0)), mode='constant')

# Split the data into training and validation sets
X_train_padded, X_val_padded = train_test_split(padded_data, test_size=0.3, shuffle=False)


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, UpSampling3D, Add
from tensorflow.keras.models import Model

# Define ResNet block
def resnet_block(input_tensor, filters, kernel_size=(3, 3, 3), strides=(1, 1, 1)):
    x = Conv3D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = Conv3D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Adjust the shortcut connection
    if strides != (1, 1, 1) or input_tensor.shape[-1] != filters:
        shortcut = Conv3D(filters, (1, 1, 1), strides=strides, padding='same')(input_tensor)
    else:
        shortcut = input_tensor
    
    x = Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Define the model architecture
input_shape_padded = (4, 42, 102, 3)
inputs = Input(shape=input_shape_padded)

# Encoder
x = resnet_block(inputs, 32)
x = resnet_block(x, 64)
encoded = resnet_block(x, 128, strides=(2, 2, 2))

# Decoder
x = UpSampling3D((2, 2, 2))(encoded)
x = resnet_block(x, 64)
x = UpSampling3D((1, 1, 1))(x)
x = resnet_block(x, 32)
decoded = Conv3D(3, (3, 3, 3), activation='sigmoid', padding='same')(x)

# Compile the autoencoder
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
history = autoencoder.fit(X_train_padded, X_train_padded, epochs=20, batch_size=32, validation_data=(X_val_padded, X_val_padded))


Epoch 1/20
106/129 [=======================>......] - ETA: 15:55 - loss: 0.0037

In [15]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 8, 4, 42, 102, 3)]   0         []                            
                                                                                                  
 conv3d_22 (Conv3D)          (None, 8, 4, 42, 102, 32)    2624      ['input_5[0][0]']             
                                                                                                  
 batch_normalization_16 (Ba  (None, 8, 4, 42, 102, 32)    128       ['conv3d_22[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_12 (Activation)  (None, 8, 4, 42, 102, 32)    0         ['batch_normalization_16[0

                                                                                                  
 conv3d_33 (Conv3D)          (None, 16, 4, 42, 51, 64)    8256      ['up_sampling3d[0][0]']       
                                                                                                  
 add_11 (Add)                (None, 16, 4, 42, 51, 64)    0         ['batch_normalization_23[0][0]
                                                                    ',                            
                                                                     'conv3d_33[0][0]']           
                                                                                                  
 activation_19 (Activation)  (None, 16, 4, 42, 51, 64)    0         ['add_11[0][0]']              
                                                                                                  
 up_sampling3d_1 (UpSamplin  (None, 16, 4, 42, 51, 64)    0         ['activation_19[0][0]']       
 g3D)     